
# 📊 Sales Funnel Metrics for GTM & Pricing Analytics

This section demonstrates my ability to calculate key sales funnel and go-to-market (GTM) efficiency metrics using Python. 
These metrics are fundamental for forecasting, evaluating sales performance, and informing pricing strategy.

The calculation for these metrics typically involves the concept of **conversion rate**, which is generally:
$$\text{Conversion Rate} = \frac{\text{Number of entities moving to the next stage}}{\text{Number of entities in the current stage}}$$

### Table of Contents:
1. Lead-to-Opportunity Conversion Rate (L2O) 
2. Opportunity-to-Win Conversion Rate (O2W) / Win Rate 
3. Sales Cycle Length (Time to Close) 
4. Average Deal Size (or Average Annual Contract Value - ACV) 
5. Pipeline Coverage Ratio


In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta

***

## 1. Lead-to-Opportunity Conversion Rate (L2O)

Measures the efficiency of converting raw leads into qualified opportunities.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | Gauges the effectiveness of **marketing efforts** and initial lead qualification (SDR/BDR teams). |
| **Formula** | $$\frac{\text{COUNT(Opportunities Created)}}{\text{COUNT(Total Leads Created)}}$$ |
| **Required Data Fields** | `lead_id`, `is_opportunity_created` (Boolean/Flag), `lead_creation_date` |

In [0]:
df_leads = pd.read_csv('/Workspace/Users/olesia.tankersley@thermofisher.com/Go-To-Market-Analytics/lead_funnel_data.csv',
                       parse_dates = ['lead_creation_date', 'opportunity_creation_date'])

In [0]:
df_leads.head(5)

,lead_id,lead_creation_date,is_opportunity_created,opportunity_creation_date
0,L-1,2025-01-01,1,2025-01-02
1,L-10,2025-01-19,1,2025-01-24
2,L-100,2025-06-18,0,NaT
3,L-101,2025-06-19,0,NaT
4,L-102,2025-06-20,1,2025-06-25


In [0]:
#Python code to Calculate Lead-to-Opportunity (L2O) Conversion Rate by Month
df_leads.groupby(df_leads['lead_creation_date'].dt.month).agg({'lead_id': 'count', 'is_opportunity_created': 'sum'}).reset_index().rename(columns = {'lead_creation_date':'month', 'lead_id': 'num of leads', 'is_opportunity_created': 'num of opps'})

,month,num of leads,num of opps
0,1,15,6
1,2,15,7
2,3,20,8
3,4,20,6
4,5,20,10
5,6,20,9
6,7,18,8
7,8,12,3
8,9,15,6
9,10,20,8


In [0]:
#Python code to Calculate Lead-to-Opportunity (L2O) Conversion Rate by Month
df_leads.groupby(df_leads['lead_creation_date'].dt.month).agg({'lead_id': 'count', 'is_opportunity_created': 'sum'})\
  .apply(lambda x: x['is_opportunity_created']/x['lead_id'], axis = 1).reset_index().rename(columns = {'lead_creation_date':'month', 0: 'L2O'})

,month,L2O
0,1,0.400000
1,2,0.466667
2,3,0.400000
3,4,0.300000
4,5,0.500000
5,6,0.450000
6,7,0.444444
7,8,0.250000
8,9,0.400000
9,10,0.400000


***
## 2. Opportunity-to-Win Conversion Rate (Win Rate)

Measures the percentage of closed opportunities that result in a new customer (a "win").

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | Core indicator of **sales team effectiveness** and market competitiveness (product/pricing). |
| **Formula** | $$\frac{\text{COUNT(Won Deals)}}{\text{COUNT(Total Closed Opportunities)}}$$ |
| **Required Data Fields** | `opportunity_id`, `deal_status` (must include 'Won' and 'Lost'), `closed_date` |

In [0]:
df_opportunity = pd.read_csv('/Workspace/Users/olesia.tankersley@thermofisher.com/Go-To-Market-Analytics/opportunity_funnel_data.csv',
                             parse_dates = ['opportunity_creation_date', 'closed_date'])

In [0]:
df_opportunity['Is_Won'] = df_opportunity['deal_status'] == 'Won'

In [0]:
df_opportunity.head(5)

,opportunity_id,opportunity_creation_date,deal_value,closed_date,deal_status,Is_Won
0,OPP-202501-01,2025-01-01,15000,2025-01-15,Won,True
1,OPP-202501-02,2025-01-02,25000,2025-01-20,Lost,False
2,OPP-202501-03,2025-01-03,30000,2025-01-25,Won,True
3,OPP-202501-04,2025-01-05,12000,2025-01-18,Lost,False
4,OPP-202501-05,2025-01-07,40000,2025-02-01,Won,True


In [0]:
#Python code to Calculate Opportunity-to-Win Conversion Rate (Win Rate) by Month
df_opportunity.groupby(df_opportunity['opportunity_creation_date'].dt.month).agg({'opportunity_id': 'count', 'Is_Won': 'sum'}).reset_index().rename(columns = {'opportunity_creation_date': 'month', 'opportunity_id': 'num of opportunities', 'Is_Won': 'num of wins'})

,month,num of opportunities,num of wins
0,1,20,8
1,2,20,6
2,3,18,7
3,4,21,6
4,5,21,8
5,6,15,5
6,7,19,6
7,8,26,9
8,9,17,7
9,10,23,6


In [0]:
#Python code to Calculate Opportunity-to-Win Conversion Rate (Win Rate) by Month
df_opportunity.groupby(df_opportunity['opportunity_creation_date'].dt.month).agg({'opportunity_id': 'count', 'Is_Won': 'sum'}).apply(lambda x: x['Is_Won']/x['opportunity_id'], axis=1).reset_index().rename(columns = {'opportunity_creation_date': 'month', 0: 'O2W'})

,month,O2W
0,1,0.400000
1,2,0.300000
2,3,0.388889
3,4,0.285714
4,5,0.380952
5,6,0.333333
6,7,0.315789
7,8,0.346154
8,9,0.411765
9,10,0.260870


***
## 3. Sales Cycle Length (Time to Close)

Measures the average duration (in days) from the opportunity creation to its closure.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | Shorter cycles improve forecasting accuracy and **cash flow**. Informs sales process optimization. |
| **Formula** | $$\text{AVG}(\text{Closed Date} - \text{Opportunity Creation Date})$$ |
| **Required Data Fields** | `opportunity_id`, `opportunity_creation_date`, `closed_date`, `deal_status` (Filter for closed deals only) |



In [0]:
#Python code to calculate Sales Cycle Length (Time to Close) by Month
df_opportunity['num of days'] = df_opportunity['closed_date'] - df_opportunity['opportunity_creation_date']

In [0]:
#Python code to calculate Sales Cycle Length (Time to Close) by Month. All deals.
df_opportunity.groupby(df_opportunity['opportunity_creation_date'].dt.month)['num of days'].mean().dt.days.reset_index().rename(columns = {'opportunity_creation_date':'month', 'num of days': 'avg days'})

,month,avg days
0,1,25
1,2,29
2,3,29
3,4,31
4,5,37
5,6,29
6,7,30
7,8,48
8,9,32
9,10,40


In [0]:
#Python code to calculate Sales Cycle Length (Time to Close) by Month. Only won deals. 
df_opportunity[df_opportunity['Is_Won'] == True].groupby(df_opportunity['opportunity_creation_date'].dt.month)['num of days'].mean().dt.days.reset_index().rename(columns = {'opportunity_creation_date':'month', 'num of days': 'avg days'})

,month,avg days
0,1,25
1,2,29
2,3,30
3,4,33
4,5,35
5,6,30
6,7,34
7,8,42
8,9,30
9,10,39


***
## 4. Average Deal Size (Average ACV)

The average Annual Contract Value (ACV) of all closed-won deals within a period.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | A primary **pricing analytic**. Tracks shifts in customer segmentation and success of pricing tiers/upsells. |
| **Formula** | $$\frac{\text{SUM(Deal Value of Won Deals)}}{\text{COUNT(Won Deals)}}$$ |
| **Required Data Fields** | `opportunity_id`, `deal_value` (or `acv`), `deal_status` (Must be 'Won'), `closed_date` |



In [0]:
#Python code to calculate Average Deal Size (Average ACV) by Month
df_opportunity[df_opportunity['Is_Won'] == True].groupby(df_opportunity['opportunity_creation_date'].dt.month)['deal_value'].sum().reset_index().rename(columns = {'opportunity_creation_date':'month', 'deal_value': 'Average ACV'})

,month,Average ACV
0,1,214000
1,2,146000
2,3,178000
3,4,152000
4,5,213000
5,6,136000
6,7,119000
7,8,238000
8,9,189000
9,10,152000


***
## 5. Pipeline Coverage Ratio

Compares the value of the active sales pipeline against the revenue target for a period.

| Detail | Description |
| :--- | :--- |
| **Why it Matters** | A **GTM health metric** that informs resource allocation and risk. A common target is 3x to 4x coverage. |
| **Formula** | $$\frac{\text{SUM(Value of Active Opportunities)}}{\text{Sales Revenue Target}}$$ |
| **Required Data Fields** | `opportunity_id`, `deal_value` (Forecasted), `stage` (Must exclude 'Closed' stages), **External Data: Revenue Target** |

In [0]:
df_pipeline = pd.read_csv('/Workspace/Users/olesia.tankersley@thermofisher.com/Go-To-Market-Analytics/active_pipeline_data.csv',
                          parse_dates = ['estimated_close_date'])

In [0]:
#target is set for the month
target = {1: 100000, 2: 100000, 3: 100000, 4: 100000, 5: 100000, 6: 200000, 7: 200000, 8: 200000, 9: 200000, 10: 300000, 11: 300000, 12: 400000}
df_pipeline['target'] = df_pipeline['estimated_close_date'].dt.month.map(target)

In [0]:
df_pipeline.head()

,opportunity_id,current_stage,forecasted_deal_value,estimated_close_date,target,year,month
0,PIPE-2025-01,Negotiation,55000,2025-10-15,300000,2025,10
1,PIPE-2025-02,Proposal,28000,2025-10-25,300000,2025,10
2,PIPE-2025-03,Discovery,17000,2025-11-01,300000,2025,11
3,PIPE-2025-04,Negotiation,48000,2025-11-10,300000,2025,11
4,PIPE-2025-05,Discovery,22000,2025-11-15,300000,2025,11


In [0]:
df_pipeline['year'] = df_pipeline['estimated_close_date'].dt.year
df_pipeline['month'] = df_pipeline['estimated_close_date'].dt.month

In [0]:
#df_pipeline['current_stage'].unique()

In [0]:
#Python code to calculate Pipeline Coverage Ratio by Month
df_pipeline.loc[~(df_pipeline['current_stage'].str.contains('Closed'))].groupby(['year', 'month']).agg({'forecasted_deal_value': 'sum', 'target': 'mean'})

forecasted_deal_value    target
year month                                 
2025 10                     83000  300000.0
     11                    252000  300000.0
     12                    424000  400000.0
2026 1                     556000  100000.0
     2                     545000  100000.0
     3                     756000  100000.0
     4                     464000  100000.0
     5                     798000  100000.0
     6                     632000  200000.0
     7                     430000  200000.0
     8                     673000  200000.0
     9                     751000  200000.0
     10                    431000  300000.0
     11                    624000  300000.0
     12                    597000  400000.0

In [0]:
#Python code to calculate Pipeline Coverage Ratio by Month
df_pipeline.loc[~(df_pipeline['current_stage'].str.contains('Closed'))].groupby(['year', 'month']).agg({'forecasted_deal_value': 'sum', 'target': 'mean'})\
  .apply(lambda x: round(x['forecasted_deal_value']/x['target'],2), axis = 1)

year  month
2025  10       0.28
      11       0.84
      12       1.06
2026  1        5.56
      2        5.45
      3        7.56
      4        4.64
      5        7.98
      6        3.16
      7        2.15
      8        3.36
      9        3.76
      10       1.44
      11       2.08
      12       1.49
dtype: float64